In [1]:
# !ls -hl|grep csv
import pandas as pd
import numpy as np
import os
from time import time, ctime

from sklearn.metrics import accuracy_score, classification_report, classification, confusion_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, StratifiedShuffleSplit

dim=lambda *x: [i.shape for i in x]

In [2]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib

from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

K.set_session(sess)

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5723941342426228104
]


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import os
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [4]:
df = pd.read_csv('./goden_before_label2.csv')
df = df.fillna(0)
print (df.shape)

df['summary_completion_rate_4'] = df['summary_completion_rate_4'].apply(lambda x: str(x).replace('%','')).astype(int)
df['summary_on_budget_4'] = df['summary_on_budget_4'].apply(lambda x: str(x).replace('%','')).astype(int)
df['summary_on_time_4'] = df['summary_on_time_4'].apply(lambda x: str(x).replace('%','')).astype(int)
df['summary_rehire_rate_4'] = df['summary_rehire_rate_4'].apply(lambda x: str(x).replace('%','')).astype(int)

(34219, 853)


In [5]:
X=df.iloc[:, 2:]
y=df.iloc[:, 1].map({'male':1, 'female':0}).values

In [6]:
%%time
cat_index = [list(X.columns).index(i) for i in X.select_dtypes(include=['O']).columns]
print (len(cat_index))

30
CPU times: user 15 ms, sys: 2 ms, total: 17 ms
Wall time: 15.4 ms


In [7]:
X.iloc[:,cat_index].head()

,username_split_predict,last,last_two,first,first2,about.company,about.hourly_rate,about.primary_language,Month,city_3,...,identity_verified,payment_verified,phone_verified,profile_complete,country_5,timezone_5,pred_about.username_sentence,pred_about.username,pred_about.display_name,pred_about.public_name
0,male,m,nm,a,a1,0,None,en,December,Ballymena,...,False,True,False,True,UK,Europe/London,male,male,male,male
1,unknow,e,ce,a,a1,a1myspace,None,en,July,Turramurra,...,False,False,False,True,AU,Australia/Sydney,unknow,unknow,unknow,unknow
2,female,m,om,a,a1,A1x.com,None,en,August,Littleton,...,False,False,False,False,US,America/Chicago,female,female,female,female
3,unknow,n,en,a,a2,a8 group,None,en,March,New York,...,False,False,False,True,US,America/New_York,unknow,unknow,unknow,unknow
4,unknow,s,ks,a,a2,0,None,en,May,Ann Arbor,...,False,False,False,True,US,America/New_York,unknow,unknow,unknow,unknow


In [8]:
# for i in X.iloc[:,cat_index]:
#     print i, len(X[i].unique())

In [9]:
%%time
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Labels={}

for i in X.select_dtypes(include=['O']).columns:
    exec("labelencoder_{} = LabelEncoder()".format(i.replace('.','_')))
    exec("X['{}'] = labelencoder_{}.fit_transform(X['{}'])".format(i,i.replace('.','_'),i))
    exec("Labels['labelencoder_{}']=labelencoder_{}".format(i.replace('.','_'), i.replace('.','_')))

XX=X.copy()
print (len(Labels))

30
CPU times: user 504 ms, sys: 43 ms, total: 547 ms
Wall time: 545 ms


In [14]:
df9 = pd.concat([df.iloc[:,0],pd.Series(y, name='Gender'),X],axis=1)
df9.shape

(34219, 853)

In [15]:
df9.to_csv('prepared_for_GPU.csv', index=False)

In [17]:
print cat_index

[0, 1, 2, 3, 4, 747, 748, 749, 757, 766, 767, 768, 769, 770, 771, 774, 775, 836, 837, 838, 839, 840, 841, 842, 843, 846, 847, 848, 849, 850]


In [73]:
%%time
print X.shape
onehotencoder = OneHotEncoder(categorical_features = cat_index)
X = onehotencoder.fit_transform(X).toarray()
print X.shape

(34219, 851)
(34219, 37603)
CPU times: user 1.86 s, sys: 1.99 s, total: 3.85 s
Wall time: 3.84 s


In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)
print X.shape

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=7)
dim(X_train, X_test, y_train, y_test)

[(34482, 4491), (14778, 4491), (34482,), (14778,)]

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print dim(X_train,X_test, y_train, y_test)

[(34482, 4491), (14778, 4491), (34482,), (14778,)]
CPU times: user 2.5 s, sys: 1.29 s, total: 3.78 s
Wall time: 3.78 s


In [13]:
classifier = Sequential()
# first hidden layer
classifier.add(Dense(units = 256, 
                     input_dim=4491,
                     kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.5))
# second hidden layer
classifier.add(Dense(units = 128,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.4))
# second hidden layer
classifier.add(Dense(units = 64,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.3))
# thrid hidden layer
classifier.add(Dense(units = 48,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.2))
# thrid hidden layer
classifier.add(Dense(units = 32,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 24,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 256,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.5))
# # ouput layer
classifier.add(Dense(units = 1,  kernel_initializer='uniform', activation='sigmoid'))
# compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               1149952   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 48)                3120      
__________

In [14]:
from keras.callbacks import Callback, EarlyStopping
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=1)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [15]:
stopping = EarlyStopping(monitor='acc', min_delta=0,
                              patience=6, verbose=1, mode='auto')

In [16]:
# fitting ANN with training set
classifier.fit(X_train, y_train, 
               batch_size=256, epochs=16,
               validation_data=(X_test, y_test),
          callbacks=[TestCallback((X_test, y_test))])
#           callbacks=[stopping])
    


Train on 34482 samples, validate on 14778 samples
Epoch 1/16
14720/14778 [============================>.] - ETA: 0s
Testing loss: 0.460867927287, acc: 0.81979970222

34482/34482 [==============================] - 6s - loss: 0.5026 - acc: 0.8165 - val_loss: 0.4609 - val_acc: 0.8198
Epoch 2/16
14080/14778 [===========================>..] - ETA: 0s
Testing loss: 0.474614388352, acc: 0.81979970222

34482/34482 [==============================] - 6s - loss: 0.4400 - acc: 0.8200 - val_loss: 0.4746 - val_acc: 0.8198
Epoch 3/16
14112/14778 [===========================>..] - ETA: 0s
Testing loss: 0.506805902135, acc: 0.81979970222

34482/34482 [==============================] - 6s - loss: 0.4154 - acc: 0.8200 - val_loss: 0.5068 - val_acc: 0.8198
Epoch 4/16
14208/14778 [===========================>..] - ETA: 0s
Testing loss: 0.488993572574, acc: 0.81337122746

34482/34482 [==============================] - 6s - loss: 0.3945 - acc: 0.8243 - val_loss: 0.4890 - val_acc: 0.8134
Epoch 5/16
14688/14778

KeyboardInterrupt: 

In [19]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred> 0.5)

print accuracy_score(y_test, y_pred)

plot_confusion_matrix(confusion_matrix(y_test, y_pred), ['female','male'])

NameError: name 'classifier' is not defined